In [1]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11471130962457332642, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 48037888
 locality {
   bus_id: 1
 }
 incarnation: 10827895117604374774
 physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0"]

In [2]:
from sklearn.model_selection import cross_val_score, KFold
from keras.models import Sequential
from sklearn.metrics import accuracy_score
import pandas as pd
from keras.optimizers import RMSprop
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
K.set_image_dim_ordering('th')
seed = 42

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5105)
/home/cuda/.local/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [3]:

import importlib
import os

def set_keras_backend(backend):

    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backendhttp://localhost:8888/notebooks
        importlib.reload(K)
        assert K.backend() == backend

set_keras_backend("tensorflow")


Using TensorFlow backend.


In [17]:
import numpy as np
from PIL import Image

In [4]:
data = pd.read_csv('keys.txt', delimiter="(",  header=None)

In [5]:
for i in range(5):
    data[i] = data[i].astype(str).str[:-1]

In [6]:
data.columns = ["image","Head","left_hand","steering_wheel","right_hand"]

In [7]:
data["Head_x"], data["Head_y"] = data['Head'].str.split(',', 1).str
data["left_hand_x"], data["left_hand_y"] = data['left_hand'].str.split(',', 1).str
data["steering_wheel_x"], data["steering_wheel_y"] = data['steering_wheel'].str.split(',', 1).str
data["right_hand_x"], data["right_hand_y"] = data['right_hand'].str.split(',', 1).str

In [8]:
data.drop("Head",axis=1,inplace=True)
data.drop("left_hand",axis=1,inplace=True)
data.drop("steering_wheel",axis=1,inplace=True)
data.drop("right_hand",axis=1,inplace=True)
data.head()

,image,Head_x,Head_y,left_hand_x,left_hand_y,steering_wheel_x,steering_wheel_y,right_hand_x,right_hand_y
0,img_56.jpg,192,178,461,126,443,219,587,321
1,img_63.jpg,333,135,636,56,581,171,637,383
2,img_118.jpg,242,134,539,161,540,240,633,398
3,img_120.jpg,163,132,479,110,440,213,592,304
4,img_138.jpg,192,185,514,105,490,213,588,407


In [9]:
def key_model():
    model=Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(3,150,150)))
    model.add(Activation('relu'))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))

    # the model so far outputs 3D feature maps (height, width, features)
    model.add(Flatten()) # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64)) # 64 neurons
    model.add(Activation('relu'))
    model.add(Dense(64)) # 64 neurons
    model.add(Activation('relu'))
    model.add(Dense(32)) # 64 neurons
    model.add(Activation('relu'))

    # output layer: get 8 coordinate values
    model.add(Dense(8))
    model.compile(loss='mean_squared_error', optimizer=RMSprop())
    return model

In [10]:
data.values

array([['img_56.jpg', '192', '178', ..., '219', '587', '321'],
       ['img_63.jpg', '333', '135', ..., '171', '637', '383'],
       ['img_118.jpg', '242', '134', ..., '240', '633', '398'],
       ..., 
       ['img_20722.jpg', '206', '148', ..., '217', '592', '336'],
       ['img_20750.jpg', '216', '108', ..., '164', '625', '291'],
       ['img_20763.jpg', '288', '92', ..., '170', '628', '339']], dtype=object)

In [11]:
X = data["image"].values

In [13]:
data.drop("image",axis=1, inplace=True)

In [14]:
data.head()

,Head_x,Head_y,left_hand_x,left_hand_y,steering_wheel_x,steering_wheel_y,right_hand_x,right_hand_y
0,192,178,461,126,443,219,587,321
1,333,135,636,56,581,171,637,383
2,242,134,539,161,540,240,633,398
3,163,132,479,110,440,213,592,304
4,192,185,514,105,490,213,588,407


In [15]:
Y = data.values

In [18]:
x = np.array([np.array(Image.open(fname).resize((150,150),Image.ANTIALIAS)) for fname in "key_data/"+X])

In [19]:
x.shape

(499, 150, 150, 3)

In [27]:
img = Image.fromarray(x[0], 'RGB')

In [28]:
img.save('out.png')